In [85]:
import sys
sys.path.append('/Users/calvinsmith/dufresne_lab/multicomponent-mixtures-main')
import multicomp as mm
import numpy as np
sys.path.append('/Users/calvinsmith/dufresne_lab/lab_work/modules')
import generate as gen

import importlib
from importlib import reload
importlib.reload(gen)
reload(gen)

<module 'generate' from '/Users/calvinsmith/dufresne_lab/lab_work/modules/generate.py'>

In [86]:

#Generate combinations with (num_comp, step_size)
#Threshold function, gets rid of all rows with values less than threshold

num_comps = 7


test_comb = gen.generate_combinations_2(num_comps,0.05)
test_comb = gen.replace_negative(test_comb,0)
test_comb = gen.thresh_2(test_comb,0.02)
print(test_comb)

combinations = test_comb


[[0.05 0.05 0.05 ... 0.05 0.05 0.7 ]
 [0.05 0.05 0.05 ... 0.05 0.1  0.65]
 [0.05 0.05 0.05 ... 0.05 0.15 0.6 ]
 ...
 [0.65 0.05 0.1  ... 0.05 0.05 0.05]
 [0.65 0.1  0.05 ... 0.05 0.05 0.05]
 [0.7  0.05 0.05 ... 0.05 0.05 0.05]]


# Obtain coexisting phases for an arbitrary interaction matrix and average compositions
Note: package assumes the existence of an inert solvent, determined by the incompressibility condition.

## system settings

In [87]:
# an arbitrary symmetric matrix of 4 components + 1 inert solvent.
# The interaction between components and the solvent is assume to be zero.
# Note that the diagonal elements are not necessary to be zero.
# With non-zero diagonal elements, the system will be effective interactions between solvent and other components.
'''chi_matrix = [
    [0.0, 4.0, 5.0, 6.0],
    [4.0, 0.0, 7.0, 6.0],
    [5.0, 7.0, 0.0, 3.0],
    [6.0, 6.0, 3.0, 0.0],
]
'''

chi_matrix = np.array([[0, 2.2, 2.2], [0, 0, 2.2], [0, 0, 0]])
chi_matrix = chi_matrix + chi_matrix.T

# average compositions of the entire system
# the volume fraction of the solvent is determined by incompressibility
phi_vector = [0.33, 0.33, 0.33]

## Create some compartments

In [88]:
# number of phases that the system will generate
num_compartments = 64 # a overkill but safe choice for 4+1 components
# create the Flory-Huggins free energy
fh_energy = mm.FloryHuggins(np.array(chi_matrix))
# generate a collection of compartments
# we generate compartments that all of them share the same composition, namely the average composition, here.
# the FieldLikeRelaxationDynamics afterwards will add fluctuations to the compositions itself.


phis = [phi_vector] * num_compartments
mixture = mm.MultiphaseVolumeSystem(
    fh_energy, 
    np.array(phis), # compositions of each compartment
    np.ones(num_compartments) / num_compartments # relative volume of each compartment
)


## Hyperparameters 

In [89]:
dynamics_params = {
    "initialization_random_std": 5,  # how random the initial guess is
    "acceptance_Js": 0.0002, # how fast the relative volumes are evolved
    "acceptance_omega": 0.002, # how fast the compositions are evolved
}

evolve_params = {
    "t_range": 10000,  # maximum iteration steps
    "dt": 1,  # always unity for FieldLikeRelaxationDynamics
    "interval": 10000,  # the frequency for checking convergence
    "tolerance": 1e-5,  # criteria for determining convergence
    "progress": True,
    "save_intermediate_data": False,
}

## Create the dynamics and find the coexisting states

In [90]:
dynamics = mm.FieldLikeRelaxationDynamics(mixture, parameters=dynamics_params)
ts, result = dynamics.evolve(**evolve_params)

## Print and Check

In [91]:
unique_phases = result.get_clusters()
print(len(unique_phases), "phases are found")
print("Compositions are ")
print(unique_phases)
print("Convergence is (lower the better) ")
print(result.entropy_production)



3 phases are found
Compositions are 
[[0.3257866  0.32594748 0.33825698]
 [0.3258765  0.33825846 0.32585593]
 [0.3337413  0.32823637 0.32803119]]
Convergence is (lower the better) 
0.06936749113499993


# Test Functions

In [92]:
gen.get_num_of_phases_fast(phi_vector,fh_energy)


3

In [95]:
from tqdm.notebook import tqdm
print("start")

combinations = gen.generate_combinations_2(3,0.1)
combinations = np.array([row for row in combinations if 0 not in row])

combinations -= 0.01
combinations = gen.replace_negative(combinations,0.01)

ternary_matrix = np.empty((combinations.shape[0], 2), dtype=object)
#print(combinations)

print("Init ternary_matrix")
num_compartments = 64

for i, row in enumerate(tqdm(combinations)):
    print("row " , i, row)
    ternary_matrix[i, 0] = row
   
    # Apply the arbitrary function to the row and add the result to the second column
    fh_energy = mm.FloryHuggins(np.array(chi_matrix))
    phis = [row] * num_compartments
    mixture = mm.MultiphaseVolumeSystem(
                    fh_energy, 
                    np.array(phis), # compositions of each compartment
                    np.ones(num_compartments) / num_compartments # relative volume of each compartment
                    )
    ternary_matrix[i, 1] = gen.get_num_of_phases(row,fh_energy)
    print(ternary_matrix[i,1])

    


start
Init ternary_matrix


row  0 [0.09 0.09 0.79]
1
row  1 [0.09 0.19 0.69]
1
row  2 [0.09 0.29 0.59]
1
row  3 [0.09 0.39 0.49]
1
row  4 [0.09 0.49 0.39]
1
row  5 [0.09 0.59 0.29]
1
row  6 [0.09 0.69 0.19]
1
row  7 [0.09 0.79 0.09]
1
row  8 [0.19 0.09 0.69]
1
row  9 [0.19 0.19 0.59]
1
row  10 [0.19 0.29 0.49]
1
row  11 [0.19 0.39 0.39]
1
row  12 [0.19 0.49 0.29]
1
row  13 [0.19 0.59 0.19]
1
row  14 [0.19 0.69 0.09]
1
row  15 [0.29 0.09 0.59]
1
row  16 [0.29 0.19 0.49]
1
row  17 [0.29 0.29 0.39]
1
row  18 [0.29 0.39 0.29]
1
row  19 [0.29 0.49 0.19]
1
row  20 [0.29 0.59 0.09]
1
row  21 [0.29 0.69 0.01]
2
row  22 [0.39 0.09 0.49]
1
row  23 [0.39 0.19 0.39]
1
row  24 [0.39 0.29 0.29]
1
row  25 [0.39 0.39 0.19]
1
row  26 [0.39 0.49 0.09]
1
row  27 [0.39 0.59 0.01]
2
row  28 [0.49 0.09 0.39]
1
row  29 [0.49 0.19 0.29]
1
row  30 [0.49 0.29 0.19]
1
row  31 [0.49 0.39 0.09]
1
row  32 [0.59 0.09 0.29]
1
row  33 [0.59 0.19 0.19]
1
row  34 [0.59 0.29 0.09]
1
row  35 [0.59 0.39 0.01]
2
row  36 [0.69 0.09 0.19]
1
row  37 [0.

In [96]:
reload(mm)


ternary_matrix = np.empty((combinations.shape[0], 2), dtype=object)

num_compartments = 64
#row = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.7 ]# composition?
row = [0.2,0.1,0.3,0.2]
ternary_matrix[i, 0] = row

# Apply the arbitrary function to the row and add the result to the second column
fh_energy = mm.FloryHuggins(np.array(chi_matrix))
phis = [row] * num_compartments
mixture = mm.MultiphaseVolumeSystem(
                fh_energy, 
                np.array(phis), # compositions of each compartment
                np.ones(num_compartments) / num_compartments # relative volume of each compartment
                )
ternary_matrix[i, 1] = gen.get_num_of_phases(row,fh_energy)
print(ternary_matrix[i,1])




AssertionError: 

In [82]:
fh_energy.shape, phis.shape, num_compartments

AttributeError: 'FloryHuggins' object has no attribute 'shape'

# Plot Ternary Diagram

In [97]:
import matplotlib.pyplot as plt
import mpltern  # Import the mpltern package for ternary plots
def plot_ternary(data):

    ternary_components = np.array([item[0].flatten() for item in data])
    values = np.array([item[1] for item in data])

    # Create ternary plot
    fig, tax = plt.subplots(subplot_kw=dict(projection='ternary'))

    # Scatter plot where color depends on the value
    sc = tax.scatter(ternary_components[:, 0], ternary_components[:, 1], ternary_components[:, 2], c=values, cmap='viridis')

    # Add color bar
    plt.colorbar(sc, label='Value')

    # Setting labels for ternary axes
    tax.set_tlabel("Component 1")
    tax.set_llabel("Component 2")
    tax.set_rlabel("Component 3")

    # Set title and other plot labels if necessary
    plt.title("Ternary Phase Diagram")

    plt.show()




In [98]:
plot_ternary(ternary_matrix)
print(ternary_matrix)
#np.savetxt("ternary_0.02_chi_2.2_.csv", ternary_matrix, delimiter=",")

AttributeError: 'NoneType' object has no attribute 'flatten'

In [159]:

# Flatten the array structure
flattened_data = np.array([np.append(subarray, value) for subarray, value in ternary_matrix])

# Absolute path where the CSV file will be saved
name = 'ternary_0.02_chi_2.2_TEST.csv'
save_path = '/Users/calvinsmith/dufresne_lab/data/' + name  # Change this to your desired path

# Save to CSV
np.savetxt(save_path, flattened_data, delimiter=",", fmt='%g')
print('done')

done


In [162]:

import pickle

# Create a sample numpy array
array_to_save = ternary_matrix

# Save the array to a pickle file
with open('/Users/calvinsmith/dufresne_lab/data/ternary_matrix_0.02_2.2.pkl', 'wb') as f:
    pickle.dump(array_to_save, f)

print("Array saved as pickle file.")


Array saved as pickle file.
